# Two-Input One-Output Model

In [1]:
from   keras.datasets import mnist
from   keras.layers import (
    Conv2D, Dense, Dropout, Flatten, Input, MaxPooling2D)
from   keras.layers.merge import concatenate
from   keras.models import Model
from   keras.utils import plot_model, to_categorical
import numpy as np

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
n_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [4]:
image_size = X_train.shape[1]
X_train = (
    np.reshape(X_train, [-1, image_size, image_size, 1]).astype('float32') 
    / 255)
X_test = (
    np.reshape(X_test, [-1, image_size, image_size, 1]).astype('float32') 
    / 255)

In [5]:
input_shape = (image_size, image_size, 1)
BATCH = 32
KERNEL = 4
DROPOUT = 0.4
N_FILTERS = 32

In [7]:
# Branch 1 of network
left_inputs = Input(shape=input_shape)
X1 = left_inputs
filters = N_FILTERS

# 3-layers of Conv2D-Dropout-MaxPooling2D
for i in range(3):
    X1 = Conv2D(filters=N_FILTERS, 
                kernel_size=KERNEL, 
                padding='same', 
                activation='relu')(X1)
    X1 = Dropout(DROPOUT)(X1)
    X1 = MaxPooling2D()(X1)
    filters *= 2
    
# Branch 2
right_inputs = Input(shape=input_shape)
X2 = right_inputs
filters = N_FILTERS

# 3-layers of Conv2D-Dropout-MaxPooling2D
for i in range(3):
    X2 = Conv2D(filters=N_FILTERS,
                kernel_size=KERNEL,
                padding='same',
                activation='relu',
                dilation_rate=2)(X2)
    X2 = Dropout(DROPOUT)(X2)
    X2 = MaxPooling2D()(X2)
    filters *= 2
    
# Merge branches
X = concatenate([X1, X2])
X = Flatten()(X)
X = Dropout(DROPOUT)(X)
outputs = Dense(n_labels, activation='softmax')(X)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
mod = Model([left_inputs, right_inputs], outputs)
plot_model(mod, to_file='cnn-y-network.png', show_shapes=True)

<img src="cnn-y-network.png" />

In [9]:
!rm cnn-y-network.png

In [10]:
mod.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 28, 28, 32)   544         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 28, 28, 32)   544         input_4[0][0]                    
__________________________________________________________________________________________________
dropout_7 

In [11]:
mod.compile(loss='categorical_crossentropy', 
            optimizer='adam', 
            metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [12]:
EPOCHS = 10

mod.fit([X_train, X_train],
        y_train,
        validation_data=([X_test, X_test], y_test),
        epochs=EPOCHS,
        batch_size=BATCH)

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 297s 5ms/step - loss: 0.2331 - acc: 0.9258 - val_loss: 0.1957 - val_acc: 0.9848
Epoch 2/10
41120/60000 [===================>..........] - ETA: 1:32 - loss: 0.0862 - acc: 0.9738

KeyboardInterrupt: 

In [ ]:
score = mod.evaluate([X_test, X_test], y_test, batch_size=BATCH)
print(f'Test Accuracy: {100. * score[1]:.4f}%')